```--- Day 18: Duet ---```

Working threads edition :)

In [1]:
import re
from time import sleep
from collections import deque

import threading

prog = [x.strip() for x in open('input.txt').readlines()]

In [2]:
class VM(object):
    def __init__(self, progl, prognum=0, part=1, debug=False):
        self.prog = progl[:]
        self.prognum = prognum
        self.part = part
        self.debug = debug
        #
        self.registers = {'p': prognum}
        self.pc = 0
        self.running = False
        self.blocked = [False]
        self.otherblocked = self.blocked # autoconnect in case we are running only one instance as per part 1
        self.snds = deque() # send buffer
        self.sndcount = 0
        self.rcvs = self.snds # autoconnect in case we are running only one instance as per part 1
        self.lock = threading.Lock()

    def connect(self, other):
        # connect the send / rcv buffer
        self.rcvs = other.snds
        # connect the deadlock detection semaphore
        self.otherblocked = other.blocked
        # ther can be only one lock
        if self.prognum != 0:
            self.lock = other.lock
        
    def run(self):
        self.running = True
        while self.running:
            self.step()
        print(f' <P{self.prognum}> stopped')
        return self
    
    linematcher = re.compile(r'^([^ ]+) ([^ ]+)( ([^ ]+))?')

    def step(self):
        try:
            if self.pc < 0:
                # fell off the front of the program
                raise IndexError
            l = self.prog[self.pc].strip()
        except IndexError:
            # fell off the end of the program
            self.running = False
            return self
        instr, op1, op2, _unused = VM.linematcher.match(l).groups()
        f, args = VM.dispatcher[instr]
        if args == 1:
            f(self, op1)
        else:
            f(self, op1, op2)
        if self.debug:
            print(f' <P{self.prognum}> {self.pc:4}: {l} | {self.registers}', flush=True)
        with self.lock: # Critical section
            # DEADLOCK detection
            if self.blocked[0] & self.otherblocked[0]: 
                self.running = False
                print(f'!! P{self.prognum} detected DEADLOCK !!')
        return self
    
    def _snd(self, op):
        with self.lock: # Critical section
            self.sndcount += 1
            self.snds.append(self.value(op))
            self.otherblocked[0] = False
        self.pc += 1

    def _rcv(self, op):
        if self.part == 1:
            # last sound played / message sent
            print(f'part 1 answer: {self.snds[-1]}')
            self.running = False
        with self.lock: # Critical section
            try:
                self.blocked[0] = False
                val = self.rcvs.popleft() # pull the oldest sound/message
            except IndexError:
                # run hot (could also sleep() a bit here)
                self.blocked[0] = True
                return # doesn't change self.pc, so rcv instruction will be retried
        self.set_reg(op, val)
        self.pc += 1
        
    def _set(self, op1, op2):
        self.set_reg(op1, self.value(op2))
        self.pc += 1

    def _add(self, op1, op2):
        self.set_reg(op1, self.value(op1) + self.value(op2))
        self.pc += 1
        
    def _mul(self, op1, op2):
        self.set_reg(op1, self.value(op1) * self.value(op2))
        self.pc += 1
        
    def _mod(self, op1, op2):
        self.set_reg(op1, self.value(op1) % self.value(op2))
        self.pc += 1
    
    def _jgz(self, op1, op2):
        if self.value(op1) > 0:
            self.pc += self.value(op2)
        else:
            self.pc += 1

    dispatcher = {'snd': (_snd, 1), # 'instr': (_instr_func, args)
                  'rcv': (_rcv, 1),
                  'set': (_set, 2),
                  'add': (_add, 2),
                  'mul': (_mul, 2),
                  'mod': (_mod, 2),
                  'jgz': (_jgz, 2)}
            
    def value(self, v):
        v = v.strip()
        try:
            return int(v)
        except ValueError:
            return self.get_reg(v)
            
    def get_reg(self, r):
        r = r.strip()
        if r in self.registers:
            return self.registers[r]
        else:
            self.registers[r] = 0
            return 0
        
    def set_reg(self, r, val):
        self.registers[r] = val

# Part 2 - with threads and added Critical Section lock()ing:-

In [3]:
prog0 = VM(prog, prognum=0, part=2, debug=False)
prog1 = VM(prog, prognum=1, part=2, debug=False)

prog0.connect(prog1)
prog1.connect(prog0)


threads = [threading.Thread(target=prog0.run), threading.Thread(target=prog1.run)]

[t.start() for t in threads]
[t.join() for t in threads] # "And now, we wait." https://www.youtube.com/watch?v=x_iPvUWyzhE

print(f'\n\npart 2 answer: {prog1.sndcount}')

!! P0 detected DEADLOCK !!!! P1 detected DEADLOCK !!

part 2 answer: 8001


 <P0> stopped <P1> stopped



In [4]:
# If they ran correctly, both programs will:
#
# * be blocked
# * be on instruction 21 ('rcv b')
# * have empty send buffers
# * will have send 8128 or 8001 times (prog0, prog1 respectively)
assert((prog0.pc, prog0.prog[prog0.pc], prog0.sndcount, prog0.blocked, len(prog0.snds)) == (21, 'rcv b', 8128, [True], 0))
assert((prog1.pc, prog1.prog[prog1.pc], prog1.sndcount, prog1.blocked, len(prog1.snds)) == (21, 'rcv b', 8001, [True], 0))